In [1]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
from bb_setup import setup_2,setup_1
import warnings
from datetime import datetime,timedelta
warnings.filterwarnings("ignore")

In [5]:
#tz = pytz.timezone('America/Sao_Paulo')
ativo = 'WIN@N'
#ativos = ['EURUSD']

pontos = 50
rate_stop = 1
rate_tp = 2
spread = 5
var_bb = 1.8

timeframe = 5

path = r'C:\Program Files\MetaTrader 5\terminal64.exe'

if not mt5.initialize(path = path):
    print("initialize() failed")
    mt5.shutdown()

df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M5, 0, 5000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})
#df = df[(df['Data'] > '2022-11-01') & (df['Data'] < '2022-12-30')]

df_ticks = pd.read_csv('./Dados/win_tick_hoje.csv')

df_acao = setup_1(df,df_ticks,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp,timeframe = timeframe)
df_acao = df_acao[(df_acao['Data'] > '2023-01-10')]

In [6]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes

print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

df_new_mes = df_acao[['Mes','resultado_binario']]
df_new_hora = df_acao[['Hora_h','resultado_binario']]

def calc_pontos(x):
    qtd_operacoes = x['qtd_operacoes']
    taxa_acerto = x['taxa_acerto']
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes

    return pontos_liquido

print('Agrupado por Hora')
df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

print('Agrupado por Mes')
df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
print(df_new_mes)
print('')


Timeframe: 5 --pontos: 50 --- rate_tp: 2 -- var_bb: 1.8
Pontos_liquido: -180.0 -- Total_operacoes: 6
Agrupado por Hora
  Hora_h  qtd_operacoes  taxa_acerto  pontos_liquido
0     11              1          0.0           -55.0
1     13              1          1.0            95.0
2     14              1          0.0           -55.0
3     15              1          0.0           -55.0
4     16              1          0.0           -55.0
5     17              1          0.0           -55.0
Agrupado por Mes
  Mes  qtd_operacoes  taxa_acerto  pontos_liquido
0  01              6     0.166667          -180.0



In [16]:
df_acao.tail(15)

,Data,Open,Close,acao,stop,tp,resultado_binario,Hora,Hora_h,Mes
4932,2023-01-09 12:50:00,109910.0,110410.0,sell,110460.0,110235.0,0.0,12:50:00,12,01
4944,2023-01-09 13:50:00,110840.0,111020.0,sell,111070.0,110845.0,0.0,13:50:00,13,01
4964,2023-01-09 15:30:00,111035.0,111160.0,sell,111210.0,110985.0,1.0,15:30:00,15,01
4981,2023-01-09 16:55:00,110775.0,110575.0,call,110525.0,110750.0,0.0,16:55:00,16,01
4992,2023-01-09 17:50:00,110200.0,109980.0,call,109930.0,110155.0,NaN,17:50:00,17,01


In [41]:
lista_media = df_ticks[df_ticks['Data'] >=  '2023-01-03 13:35:00'].head(50)['media']

In [ ]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes


df_new = pd.DataFrame([[ativo[:-5],var_bb,rate_tp,rate_stop,pontos,qtd_operacoes,taxa_acerto,pontos_bruto,pontos_liquido]],
            columns = ['ativo','var_bb','rate_tp','rate_stop','pontos','qtd_operacoes','taxa_acerto','pontos_bruto','pontos_liquido'])

df_total = pd.concat([df_total,df_new])

print(df_total.to_string())